## 7-9. 프로젝트 - Movielens 영화 추천 실습

- 이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
- 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.   
<br/>

    - 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. **MovieLens 1M Dataset** 사용을 권장합니다.
    - 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
    - 별점을 **시청횟수**로 해석해서 생각하겠습니다.
    - 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.    
<br/>   
    
- **MovieLens 1M Dataset** 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

1) wget으로 데이터 다운로드     
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 이동     
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다.      
$ unzip ml-1m.zip

----

### 1) 데이터 준비와 전처리
- Movielens 데이터는 **rating.dat** 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있음

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


**3점 이상만 남김**

In [2]:

ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')



orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


**rating 컬럼의 이름을 count로 바꿈**

In [3]:
ratings.rename(columns={'rating':'count'},inplace=True)

**영화 제목을 보기 위해 메타 데이터를 읽어오기**

In [4]:
movie_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


----

### 2) 분석하기
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [5]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


**ratings에 있는 유니크한 영화 개수**

In [7]:
ratings['movie_id'].nunique()

3628

**ratings에 있는 유니크한 사용자 수**

In [8]:
ratings['user_id'].nunique()

6039

**가장 인기있는 영화 30개(인기순)**

In [9]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

---

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [10]:
my_favorite = ['Toy Story(1995)', 'Jumanji(1995)', 'Grumpier Old Men(1995)', 'Waiting to Exhale(1995)', 'Tigerland(2000)']

# 'Tay'라는 user_id가 위 영화를 5점 평점을 주었다는 가정.
my_movielist = pd.DataFrame({'user_id': ['Tay']*5, 'movie_id': my_favorite, 'count':[5]*5})

# user_id에 'Tay'이라는 데이터가 없다면 위에 임의로 만든 my_favorite 데이터를 추가
if not ratings.isin({'user_id':['Tay']})['user_id'].any():
    ratings = ratings.append(my_movielist)

                             
ratings.tail(10)

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,Tay,Toy Story(1995),5,NaN
1,Tay,Jumanji(1995),5,NaN
2,Tay,Grumpier Old Men(1995),5,NaN
3,Tay,Waiting to Exhale(1995),5,NaN
4,Tay,Tigerland(2000),5,NaN


----

### 4) CSR matrix를 직접 만들어 봅시다.

In [11]:
data = pd.merge(ratings, movies, on='movie_id')
data.tail()

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama


In [12]:
data.drop('timestamp', axis=1, inplace=True)

In [13]:
data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [14]:
data.drop('genre', axis=1, inplace=True)
data

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975)
...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973)
836474,5854,3026,4,Slaughterhouse (1987)
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)"


In [15]:
my_favorite = ['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)', 'Waiting to Exhale (1995)', 'Tigerland (2000)']

# 'Tay'라는 user_id가 위 영화를 5점 평점을 주었다는 가정.
my_movielist = pd.DataFrame({'user_id': ['Tay']*5, 'title': my_favorite, 'count':[5]*5})

# user_id에 'Tay'이라는 데이터가 없다면 위에 임의로 만든 my_favorite 데이터를 추가
if not data.isin({'user_id':['Tay']})['user_id'].any():
    data = data.append(my_movielist)

                             
data.tail(10)

,user_id,movie_id,count,title
836473,5851,3607,5,One Little Indian (1973)
836474,5854,3026,4,Slaughterhouse (1987)
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,1360,5,Identification of a Woman (Identificazione di ...
0,Tay,NaN,5,Toy Story (1995)
1,Tay,NaN,5,Jumanji (1995)
2,Tay,NaN,5,Grumpier Old Men (1995)
3,Tay,NaN,5,Waiting to Exhale (1995)
4,Tay,NaN,5,Tigerland (2000)


In [16]:
movie_count1 = data.groupby('title')['user_id'].count()
movie_count1.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

**모델에 활용하기 위한 데이터 전처리**
- 데이터 관리를 위해 **user_id**와 **title**에 번호 붙이기
- **Indexing**

In [17]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화 indexing하는 코드 idx는 index의 약자
user_to_idx = {v:k for k, v in enumerate(user_unique)}
title_to_idx = {v:k for k, v in enumerate(title_unique)}

In [18]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['Tay'])
print(title_to_idx['Toy Story (1995)'])


6039
40


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN가 될 테니 dropna()로 제거
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):        # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!')
    data['user_id'] = temp_user_data        # data['user_id']을 인덱싱된 Series로 교체 
else:
    print('user)id column indexing Fail!')
    
    
# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!')




user_id column indexing OK!
title column indexing OK!


- **dat**a의 **user_id**와 **title**칼럼 내 값들이 모두 정수 인덱스 값으로 잘 변경 됨
- 훈련을 위한 전처리 완료

**csr_matrix 만들기**

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()



In [21]:
num_user, num_title

(6040, 3628)

In [22]:
data.count

<bound method DataFrame.count of     user_id movie_id  count  title
0         0     1193      5      0
1         1     1193      5      0
2         2     1193      4      0
3         3     1193      4      0
4         4     1193      5      0
..      ...      ...    ...    ...
0      6039      NaN      5     40
1      6039      NaN      5    513
2      6039      NaN      5   1862
3      6039      NaN      5    397
4      6039      NaN      5   2398

[836483 rows x 4 columns]>

In [40]:
csr_data = csr_matrix((data['count'].values , (data.user_id, data.title)), shape=(num_user, num_title))
csr_data


<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

----

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [24]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

**Implicit AlternatingLeastSquares 모델의 선언**

In [25]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)


In [26]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose


<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

**모델 훈련**

In [27]:
als_model.fit(csr_data_transpose)

**모델 학습 끝**     
모델이     
1) 저의 벡터와 **Toy Story (1995)** 의 벡터를 어떻게 만들고 있는지    
2) 두 벡터를 곱하면 어떤 값이 나오는지

In [28]:
Tay, Toy_Story = user_to_idx['Tay'], title_to_idx['Toy Story (1995)']
Tay_vector, Toy_Story_vector = als_model.user_factors[Tay], als_model.item_factors[Toy_Story]


In [29]:
Tay_vector

array([-0.42242175,  0.2626792 ,  0.4285717 , -0.14601637,  0.02465807,
       -0.51492465, -0.11115545, -0.0600106 ,  0.6388939 , -0.3142179 ,
       -0.13926029, -0.39055   ,  0.34796152,  0.08585892, -0.01267592,
       -0.20635314, -0.22215642,  0.02606627,  0.42129812,  0.38369137,
        0.1674104 ,  0.3899705 , -0.38706267,  0.06120895,  0.32739922,
        0.05398111,  1.0063841 , -0.02435933,  0.06070877,  0.3782162 ,
        0.7341899 , -0.2941346 , -0.01398889, -0.60390735, -0.04112607,
       -0.18059224,  0.06823461, -0.3425352 ,  0.01518982, -0.2727395 ,
       -0.51243454, -0.24194047, -0.31687507,  0.4109101 , -0.13594407,
       -0.01139427, -0.17727427, -0.16698839, -0.35600996, -0.38363737,
       -0.1243357 ,  0.03251337, -0.7857502 , -0.4965465 , -0.29743415,
        0.2724171 ,  0.10481168,  0.93181187,  0.13772364,  1.2965919 ,
       -0.25798565, -0.25848112,  0.33470058,  0.10814084,  0.04237174,
        0.28328833, -0.59205365,  0.0471031 ,  0.03343615, -0.19

In [30]:
Toy_Story_vector

array([ 8.35273508e-03,  2.13661920e-02, -1.33691849e-02, -7.94491451e-03,
       -9.40803718e-03, -2.83545535e-03, -7.74064858e-04,  1.99956764e-02,
        1.62627865e-02, -1.49817085e-02,  1.22838151e-02,  1.06287887e-02,
        7.44506205e-03, -2.79536494e-03,  1.90364420e-02,  1.88817531e-02,
        3.59769934e-03,  2.75986865e-02,  1.48287797e-02,  2.20209006e-02,
       -1.49358846e-02,  3.64242494e-02,  8.51769652e-03, -5.42832958e-03,
        2.53860205e-02,  2.42886860e-02,  2.07846500e-02,  7.86274322e-05,
        4.79571847e-03,  3.58917564e-02,  1.33175589e-03, -7.95642566e-03,
        1.04335602e-02,  7.07263220e-03,  1.90895628e-02, -1.19952150e-02,
        3.82168777e-02, -1.73582498e-03,  3.12151089e-02,  8.97331536e-03,
       -1.05023040e-02, -4.73580090e-03, -2.53036767e-02, -1.65210268e-03,
        4.56684921e-03,  1.72267482e-02, -6.12877449e-03, -3.17728445e-02,
        3.28918896e-03,  8.38238094e-03,  1.81200802e-02, -4.53779520e-03,
       -3.56590226e-02, -

**Tay과 Toy_Story를 내적하는 코드**

In [31]:
np.dot(Tay_vector, Toy_Story_vector)

0.44994375

- 1이 나와야 될 것 같은데 한참 낮은 수치인 0.49정도 나옴
- factor를 늘리거나 iterations을 늘려야 함
- 이후 진행되는 내용은 이렇게 학습된 모델을 사용 할 예정

---

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

**모델이 Men in Black에 대한 선호도 예측**

In [34]:
man_in_black = title_to_idx['Men in Black (1997)']
man_in_black_vector = als_model.item_factors[man_in_black]
np.dot(Tay_vector, man_in_black_vector)

-0.0024994165

---

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기
**비슷한 영화 찾기**
- **AlternatingLeastSquares** 클래스에 구현되어 있는 **similar_items** 메서드를 통하여 비슷한 영화를 찾습니다.

In [37]:
favorite_title = 'Shakespeare in Love (1998)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(126, 0.205912),
 (110, 0.10637674),
 (381, 0.10390924),
 (271, 0.09661914),
 (170, 0.090835154),
 (3035, 0.08692256),
 (613, 0.08665121),
 (99, 0.08125699),
 (4, 0.07905573),
 (445, 0.07580833),
 (40, 0.074785106),
 (243, 0.074619345),
 (757, 0.07357941),
 (20, 0.07151211),
 (314, 0.07052043)]

- (title의 id, 유사도) Tuple로 반환
- title의 id를 다시 title의 이름으로 매핑

In [38]:
#title_to_idx를 뒤집어, index로부터 title 이름을 얻는 dict를 생성
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Shakespeare in Love (1998)',
 'Groundhog Day (1993)',
 'Elizabeth (1998)',
 'Election (1999)',
 'Being John Malkovich (1999)',
 'For Ever Mozart (1996)',
 'Four Weddings and a Funeral (1994)',
 'American Beauty (1999)',
 "Bug's Life, A (1998)",
 'Red Violin, The (Le Violon rouge) (1998)',
 'Toy Story (1995)',
 'Ghostbusters (1984)',
 'Full Monty, The (1997)',
 'Pleasantville (1998)',
 'Much Ado About Nothing (1993)']

**몇번 더 반복해서 확인하기 위해 위 코드를 함수로 만들고 확인**

In [41]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [44]:
get_similar_title('American Beauty (1999)')

['American Beauty (1999)',
 'Being John Malkovich (1999)',
 'Election (1999)',
 'Fargo (1996)',
 'Shakespeare in Love (1998)',
 'Braveheart (1995)',
 'American Pie (1999)',
 'Saving Private Ryan (1998)',
 'Silence of the Lambs, The (1991)',
 'Paralyzing Fear: The Story of Polio in America, A (1998)']

In [45]:
get_similar_title('Star Wars: Episode IV - A New Hope (1977)')

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Raiders of the Lost Ark (1981)',
 'Back to the Future (1985)',
 'Alien (1979)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Terminator, The (1984)',
 'Matrix, The (1999)',
 'Indiana Jones and the Last Crusade (1989)']

----

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 보기
- **AlternatingLeastSquares** 클래스에 구현되어 있는 **recommend** 메서드를 통하여 제가 좋아할 만한 아티스트를 추천 받습니다. 
- **filter_already_liked_items** 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [46]:
user = user_to_idx['Tay']

# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended




[(50, 0.4117943),
 (545, 0.28097376),
 (4, 0.2721692),
 (812, 0.24821097),
 (33, 0.22209877),
 (458, 0.22069678),
 (322, 0.2149298),
 (1045, 0.21428773),
 (1130, 0.20831102),
 (173, 0.18810421),
 (596, 0.18612012),
 (1982, 0.17561899),
 (435, 0.16593198),
 (330, 0.15894699),
 (119, 0.15662074),
 (60, 0.15342686),
 (80, 0.14989203),
 (58, 0.14870837),
 (476, 0.14768223),
 (474, 0.14155626)]

In [47]:
[idx_to_title[i[0]] for i in title_recommended]

['Toy Story 2 (1999)',
 'Santa Clause, The (1994)',
 "Bug's Life, A (1998)",
 'Nutty Professor, The (1996)',
 'Aladdin (1992)',
 'Mask, The (1994)',
 'Babe (1995)',
 'Grumpy Old Men (1993)',
 'Indian in the Cupboard, The (1995)',
 'Dragonheart (1996)',
 'Hook (1991)',
 'Space Jam (1996)',
 'Godfather: Part III, The (1990)',
 'Lion King, The (1994)',
 'Twister (1996)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Stand by Me (1986)',
 'Mission: Impossible (1996)',
 'Wrong Trousers, The (1993)',
 'Chicken Run (2000)']

- **Toy Story 2** 추천 받음
- **AlternatingLeastSquares** 클래스에 구현된 **explain** 메소드를 사용하면서 기록을 남긴 데이터 중 **이 추천에 기여한 정도**를 확인 가능

In [49]:
Toy_Story2 = title_to_idx['Toy Story 2 (1999)']
explain = als_model.explain(user, csr_data, itemid=Toy_Story2)

- 이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)를 반환
- 어떤 영화들이 이 추천에 얼마나 기여하고 있는 것인지 알 수 있음

In [50]:
[(idx_to_title[i[0]],i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.38740279667419686),
 ('Jumanji (1995)', 0.022026218121805254),
 ('Waiting to Exhale (1995)', 0.016954575339336548),
 ('Grumpier Old Men (1995)', -0.0021238841006297553),
 ('Tigerland (2000)', -0.015462325929784509)]

- **Toy Story (1995)**가 가장 크게 기여
- 나머지는 미비한 영향이 있었음
- **Toy Story (1995)**가 시리즈라서 큰 영향을 받음

----

**정리 및 결과**
1. **CSR matrix**만들 때 count 변수는 넘파이 행렬임으로 사이즈를 맞춰주기 위해 **data['count']**로 설정해주어 **CSR matrix** 사이즈를 맞춤
2. 사용자 변수 **'Tay'**와 영화들관의 벡터 내적 수치를 파악하여 의미있게 형성되었는 지 확인
3. **MF Model**이 예측한 **'Tay'** 선호도 및 영화간의 유사도, 기여도가 의미있게 측정
    - **Toy Story 2 (1999)**추천 받아 내가 임의로 선호하는 5개의 영화의 유사도를 비교해본 결과 **Toy Story (1995)**의 유사도가 제일 높게 잘 나옴.
    - **Toy Story** 시리즈라서 높은 유사도를 보임
